In [392]:
import requests, csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from collections import OrderedDict
import tqdm


In [393]:
def actor_imdb(imdb_id):
    
    r2 = requests.get("https://www.imdb.com/title/"+imdb_id+"/fullcredits")
    bs2 = BeautifulSoup(r2.text, "html.parser")

    #select casting
    cast_list = bs2.find_all('table', {'class': 'cast_list'})
    if cast_list == None or cast_list == []:
        cast = [""]
        cast_code = [""]
        cast_voice = [""]
    else :
        cast_list_test = bs2.find('table', {'class': 'cast_list'}).find_all('td', {'class':'primary_photo'})
        cast_list_char = bs2.find('table', {'class': 'cast_list'}).find_all('td', {'class':'character'})
        cast_char = [text.text.strip().replace(' \n', '') for text in cast_list_char]
        
        if len(cast_list_test) >= 30 :
            cast = [str(cast_list).split('img alt="')[i].split('" class="')[0] for i in range(1,31)]
            cast_code = [str(cast_list_test).split('/name/')[i].split('/">')[0] for i in range(1,31)]
            cast_voice = ["V" if "(voice)" in text else "U" if "uncredited" in text else "" \
                          for text in cast_char[:30]]
        else :
            cast = [str(cast_list).split('img alt="')[i].split('" class="')[0] \
                    for i in range(1,len(cast_list_test)+1)]
            cast_code = [str(cast_list_test).split('/name/')[i].split('/">')[0] \
                          for i in range(1,len(cast_list_test)+1)]
            cast_voice = ["V" if "(voice)" in text else "U" if "uncredited" in text else "" \
                          for text in cast_char[:len(cast_list_test)]]
            
    return imdb_id, cast, cast_code, cast_voice
    

In [120]:
df = pd.read_csv('act_scrap.csv')

In [121]:
imdb_id = df["imdb_id"].values

In [122]:
actors = []

In [156]:
for actor in tqdm.tqdm(imdb_id[79766:]):
    actors.append(actor_imdb(actor))

100%|██████████| 6818/6818 [1:55:01<00:00,  1.01s/it]  


In [157]:
actor_df = pd.DataFrame(actors, columns = ['imdb_id', 'names', 'codes', 'status'])

In [158]:
actor_df.to_csv('temp_actor.csv')

In [159]:
imdb = list(actor_df['imdb_id'].values)
names = list(actor_df['names'].values)
codes = list(actor_df['codes'].values)
status = list(actor_df['status'].values)

## Update movie_actor_sql

In [198]:
cast_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(codes[i]))]

In [199]:
all_actors = [names[i][j] for i in range(len(imdb)) for j in range(len(names[i]))]

In [200]:
all_codes = [codes[i][j] for i in range(len(imdb)) for j in range(len(codes[i]))]

In [201]:
all_status = [status[i][j] for i in range(len(imdb)) for j in range(len(status[i]))]

In [218]:
movie_actor = {"movie_id": cast_imdb, "actor_id": all_codes, "names": all_actors, "status": all_status}

In [219]:
movie_actor_df = pd.DataFrame(movie_actor)

In [260]:
movie_actor_df.shape

(1807733, 4)

In [221]:
movie_actor_df = movie_actor_df.drop(movie_actor_df[movie_actor_df["actor_id"] == ""].index)

In [222]:
movie_actor_df = movie_actor_df.drop(movie_actor_df[movie_actor_df["status"] == "U"].index)

In [224]:
movie_actor_df = movie_actor_df.drop_duplicates()

In [269]:
movie_actor_df[:1000001].drop(["names"], axis=1).set_index("movie_id").to_csv("movie_actor_db1.csv")

In [270]:
movie_actor_df[1000001:].drop(["names"], axis=1).set_index("movie_id").to_csv("movie_actor_db2.csv")

In [257]:
actors_df = movie_actor_df.drop(["movie_id", "status"], axis=1).drop_duplicates().sort_values(["actor_id"])

In [259]:
actors_df.set_index("actor_id").to_csv("actor_db.csv")

In [371]:
back = pd.read_csv("db_backup/actor_db.csv")

In [379]:
back_ids = back.actor_id.values

In [373]:
up = pd.read_csv("update_db/up_actor.csv")

In [375]:
ids = up.actor_id.values

In [377]:
names = up.name.values

In [380]:
new_id = []
new_name = []

In [382]:
for i in range(len(ids)):
    if ids[i] not in back_ids:
        new_id.append(ids[i])
        new_name.append(names[i])

In [387]:
col = {"actor_id":new_id, "name":new_name}

In [390]:
up_actor = pd.DataFrame(col).set_index("actor_id")

In [391]:
up_actor.to_csv("update_db/up_actor.csv")